In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
import statistics
import seaborn as sns
import altair as alt
#%pip install altair vega_datasets
import sys
print(sys.executable)
print("setup complete")

In [ ]:
# import the data set that contains no nulls for all categories as representative of the entire population
import pandas as pd
Porsche_NoNulls = pd.read_csv("911_training_data.csv")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
import statistics
import seaborn as sns
import altair as alt
#%pip install altair vega_datasets
print(sys.executable)

# Now, let's see get summary statistics about the sample.  
#What is the slowest (max) and the fastest (min) in Porsche_NoNulls? 
fastest_time = np.min(Porsche_NoNulls['acceleration_0-62mph'])
slowest_time = np.max(Porsche_NoNulls['acceleration_0-62mph'])

print("The quickest 0-62mph in Porsche 911 history is",fastest_time,'s')
print("The slowest 0-62mph in Porsche 911 history is",slowest_time,'s')

# Medians tested dued to skewed acceleration times from the earliest models and the newest high performance models
P911_Median = np.median(Porsche_NoNulls['acceleration_0-62mph']) 
print("The median 0-62mph time is: {0:8.2f}".format(P911_Median))

P911_Variance = np.var(Porsche_NoNulls['acceleration_0-62mph'])
P911_StdDev = np.std(Porsche_NoNulls['acceleration_0-62mph'])

P911_Variance_round = round(P911_Variance,2)
P911_StdDev_round = round(P911_StdDev,2)

print('The variance of 0-62mph time is:',P911_Variance_round)
print('The std. dev of 0-62mph time is:',P911_StdDev_round)



In [ ]:
# Now that we have summary information across all models, let's see how these are distributed by body type.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
import statistics
import seaborn as sns
import altair as alt
#%pip install altair vega_datasets
#import sys
print(sys.executable)

Porsche_NoNulls = pd.read_csv("911_training_data.csv")

#Find correlation between start of production year and Horsepower
rounded_coeff = round(np.corrcoef(Porsche_NoNulls ['start_of_production'], Porsche_NoNulls['acceleration_0-62mph'])[0,1],2)
print("Correlation Coefficient=", rounded_coeff)

#prepare X and y for the regression model
y = Porsche_NoNulls ['acceleration_0-62mph'] #dependent variable
X = Porsche_NoNulls ['start_of_production'] #independent variable
X = sm.add_constant(X) # adds a constant term to the predictor variable

#creating a dictionary to assign different colors in the scatterplot to represent different body types
color_dict = {'Coupe': 'blue', 'Targa': 'green', 'Cabriolet': 'orange'}

#Build the regression model using OLS (ordinary least squares)
acc_speed_model = sm.OLS(y,X).fit()

#we pick 100 points equally spaced from the min to the max - this will be our 0-62mph time
X_prime = np.linspace(X['start_of_production'].min(), X['start_of_production'].max(), 100)
X_prime = sm.add_constant(X_prime) 

#now we calculate the predicted values 
y_hat = acc_speed_model.predict(X_prime)

for body_type, color in color_dict.items():
    subset = Porsche_NoNulls[Porsche_NoNulls['body_type'] == body_type]
    plt.scatter(subset['start_of_production'], subset['acceleration_0-62mph'], label=body_type, color=color)
    
plt.xlabel("start_of_production")
plt.ylabel("0-62 Acceleration Time")
plt.title("Porsche 911 acceleration times, by body type")
# Add the regression line, colored in red
plt.plot(X_prime[:, 1], y_hat, 'red', alpha=0.9)  

plt.legend(["Coupe" , "Targa", "Cabriolet"])
plt.show()

plt.show()

print(acc_speed_model.summary())
print(acc_speed_model.params)

# From the output below, it appears that the first Targa model released in the mid-70s and appeared to have
# significantly slower acceleration as opposed to its coupe counterpart. The first cabriolet was released 
# in the early 80s and had similar acceleration to the Targa 10 years earlier. However, as time has progressed
# acceleration times regardless of body style have decreased considerably over time, with Coupes being the quicker body type.
# However, there are problems in the regression model; the condition number is large and 4.576 Kurtosis suggests that 
# this is not a normally distributed dataset. 

In [ ]:
# Our R-squared value suggests that production year is a strong predictor of 0-62mph time. However, our data shows
# that there's a strong possibility of multicollinearity. Additionally, it does not appear that our data is normally 
# distributed and is skewed right. 

#Let's examine other factors that can better help explain reduction in 0-62mph times.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
import scipy as sp
import statistics
import seaborn as sns

Training_Data = pd.read_csv("911_training_data.csv")
df = pd.DataFrame(Training_Data, columns = ['start_of_production','acceleration_0-62mph','HP','torque (nm)','engine_displacement (cm^3)',
                                           'kerb_weight (kg)', 'length (mm)', 'width (mm)']) 


#Index(['generation ', 'engine', 'start_of_production',
#       'powertrain_architecture', 'body_type', 'seats', 'doors', 'fuel_type',
#       'acceleration_0-62mph', 'maximum_speed (kmh)', 'weight-to-power_ratio',
#       'weight-to-torque_ratio', 'power', 'HP', 'power_per_litre', 'torque',
#       'torque (nm)', 'engine_layout', 'engine_displacement (cm^3)',
#       'number_of_cylinders', 'engine_configuration', 'cylinder_bore (mm)',
#       'piston_stroke (mm)', 'number_of_valves_per_cylinder',
#       'engine_aspiration', 'kerb_weight (kg)', 'length (mm)', 'width (mm)',
#       'height (mm)', 'wheelbase (mm)', 'drive_wheel',
#       'number_of_gears_and_type_of_gearbox', 'front_suspension',
#       'rear_suspension', 'assisting_systems', 'steering_type',
#       'co2_emissions', 'compression_ratio', 'drag_coefficient'],
#      dtype='object')


plt.figure(figsize = (14,7))
plt.title("Correlation Matrix")

sns.heatmap(data=df.corr(method='kendall'), annot = True)

#As it might be expected, Horsepower and torque are better determinants of acceleration time.  
# It would also appear that length and width have changed every one in three years. 

In [ ]:
# Given what we know now, let's rerun the regression model with production year, horsepower, torque, length, and width 

#Index([ 'start_of_production',

#       'acceleration_0-62mph', 'maximum_speed (kmh)', 'weight-to-power_ratio',
#       'weight-to-torque_ratio', 'power', 'HP', 'power_per_litre', 'torque',
#       'torque (nm)', 'engine_layout', 'engine_displacement (cm^3)',
#       'number_of_cylinders', 'engine_configuration', 'cylinder_bore (mm)',
#       'piston_stroke (mm)', 'number_of_valves_per_cylinder',
#       'engine_aspiration', 'kerb_weight (kg)', 'length (mm)', 'width (mm)',
#       'height (mm)', 'wheelbase (mm)', 'drive_wheel',
#       'number_of_gears_and_type_of_gearbox', 'front_suspension',
#       'rear_suspension', 'assisting_systems', 'steering_type',
#       'co2_emissions', 'compression_ratio', 'drag_coefficient'],
#      dtype='object')

#'start_of_production','acceleration_0-62mph','HP','torque (nm)','engine_displacement (cm^3)',
 #                                          'kerb_weight (kg)', 'length (mm)', 'width (mm)']) 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

Porsche_NoNulls = pd.read_csv("911_training_data.csv")

#prepare X and y for the regression model
y = Porsche_NoNulls['acceleration_0-62mph'] #dependent variable
X = Porsche_NoNulls[['start_of_production', 'HP', 'torque (nm)', 'length (mm)', 'width (mm)']] #independent variables
X = sm.add_constant(X) # adds a constant term to the predictor variable

#creating a dictionary to assign different colors in the scatterplot to represent different body types
color_dict = {'Coupe': 'blue', 'Targa': 'green', 'Cabriolet': 'orange'}

#Build the regression model using OLS (ordinary least squares)
acc_speed_model = sm.OLS(y,X).fit()

#we pick 100 points equally spaced from the min to the max - this will be our 0-62mph time
X_prime = np.linspace(X[['start_of_production', 'HP', 'torque (nm)', 'length (mm)', 'width (mm)']].min(), X[['start_of_production', 'HP', 'torque (nm)', 'length (mm)', 'width (mm)']].max(), 100)
X_prime = sm.add_constant(X_prime) 

#now we calculate the predicted values 
y_hat = acc_speed_model.predict(X_prime)

for body_type, color in color_dict.items():
    subset = Porsche_NoNulls[Porsche_NoNulls['body_type'] == body_type]
    plt.scatter(subset['start_of_production'], subset['acceleration_0-62mph'], label=body_type, color=color)
    
plt.xlabel("start_of_production")
plt.ylabel("0-62 Acceleration Time")
plt.title("Porsche 911 acceleration times, by body type")
# Add the regression line, colored in red
plt.plot(X_prime[:, 1], y_hat, 'red', alpha=0.9)  

plt.legend(["Coupe" , "Targa", "Cabriolet"])
plt.show()

plt.show()

print(acc_speed_model.summary())
print(acc_speed_model.params)

# From the output below, it appears that the first Targa model released in the late 60s to early 70s and appeared to have
# significantly slower acceleration as opposed to its coupe counterpart. The first cabriolet was released 
# in the early 80s and had similar acceleration to the Targa 10 years earlier. However, as time has progressed
# acceleration times regardless of body style have decreased considerably over time, with Coupes being the quicker body type.
# However, there are problems in the regression model; the condition number is large and 4.576 Kurtosis suggests that 
# this is not a normally distributed dataset. 

# Now, let's visualize the relationship between our variables 

# Horsepower & Torque relationship with RPM 


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
import statistics
import seaborn as sns
import altair as alt
from scipy.interpolate import UnivariateSpline
#%pip install altair vega_datasets
import sys

#start_of_production     int64
#body_type              object
#power                  object
#HP                      int64
#HP RPM                  int64
#Torque (NM)             int64
#Torque RPM             object
#dtype: object


Porsche_HP_Torque = pd.read_csv("Porsche 911_HP_Torque.csv")

#Porsche_HP_Torque.head()
#Porsche_HP_Torque.dtypes

#Create 2 separate data frames: DF1 = {start_of_production & body type, HP, HP RPM} and 
#DF2 = {start_of_production & Body type, Torque, Torque RPM}


# Dataframe 1 

df1 = pd.DataFrame(Porsche_HP_Torque,columns = ['start_of_production','body_type','HP','HP RPM'])

base = alt.Chart(df1).mark_point().encode(
    x='HP:Q',
    y='HP RPM:Q',
).properties(
    width=500,
    height=500
)

alt.hconcat(
   base.encode(color='body_type:O').properties(title='Horsepower and RPM'),
)

#From the visualization below, it appears that while there is a relationship between Horsepower and RPM,
# it is not a strong relationship. The data also appears to have clusters: 1. Between 100 and 250 HP and less than
# 7000 RPM and 3. Between 300 and 550 HP and less than 8000 RPM. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
import statistics
import seaborn as sns
import altair as alt
#%pip install altair vega_datasets
import sys

#Index(['start_of_production', 'body_type', 'power', 'HP', 'HP RPM',
#       'Torque_NM', 'Torque_RPM'],
#      dtype='object')

#start_of_production     int64
#body_type              object
#power                  object
#HP                      int64
#HP RPM                  int64
#Torque_NM               int64
#Torque_RPM              int64
#dtype: object

Torque = pd.read_csv("TORQUE.CSV")

sns.relplot(
    data=Torque,
    x="Torque_NM", y="Torque_RPM",
    hue="body_type", style="body_type",
)




In [ ]:
# Now, let's see how 0-62mph times are distributed between drive_wheel and body style.
import pandas as pd
import seaborn as sns
#%pip install --upgrade seaborn


HTest1 = pd.read_csv("P911HTests.csv")

print(HTest1['drive_wheel'].unique())

sns.violinplot(data=HTest1, x="body_type", y="acceleration_0-62mph", hue="drive_wheel",
               split=True, inner="quart", fill=False,
               palette=["g", ".35"])



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
import statistics
import seaborn as sns
import altair as alt
import scipy.stats as stats
from statsmodels.stats import diagnostic
from scipy.stats import ttest_ind


# Drive Wheel and efficiency https://www.fueleconomy.gov/feg/factors.shtml

#Ho: AWD and RWD models regardless of body style have the same average fuel consumption rate. 

#Ho: AWD and RWD models regardless of body style do not have the same average fuel consumption rate. 

#Both of these are based on the Combined Liters/100KM between city and highway driving.

AWDFuelCons = [
    10, 11, 10, 9, 10, 9, 10, 10, 10, 9, 10, 8, 9, 9, 10, 8, 9, 10, 10, 10,
    9, 10, 8, 9, 10, 10, 10, 10, 9, 10, 10, 10, 10, 10, 10, 10, 9, 10, 10,
    10, 10, 10, 10, 9, 10, 12, 12, 12, 11, 12, 11, 12, 11, 11, 11, 11, 11,
    11, 12, 12, 12, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 14, 13,
    12, 12, 12, 12, 13, 13
]

df = pd.DataFrame(AWDFuelCons, columns=['values'])
df['values'] = df['values'].astype(int)

RWDFuelCons = [
    14, 10, 10, 9, 9, 9, 10, 8, 9, 8, 9, 13, 13, 13, 12, 9, 10, 8, 9, 9, 
    10, 8, 9, 9, 10, 9, 10, 9, 10, 14, 13, 13, 9, 10, 9, 10, 9, 9, 11, 11,
    11, 11, 10, 11, 14, 13, 12, 11, 11, 11, 11, 12, 12, 12, 13, 13, 13, 12,
    12, 12, 13, 12, 12, 12, 12, 12, 13, 12, 12, 13, 13, 13
]

df2 = pd.DataFrame(RWDFuelCons, columns=['values'])
df2['values'] = df2['values'].astype(int)
#print(RWDFuelCons)

# Perform t-test
ttest, pval = ttest_ind(df, df2)
print("p-value:", pval)
if pval < 0.05:
    print("We reject the null hypothesis")
else:
    print("We accept the null hypothesis")
